In [2]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from time import time
import networkx as nx
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.models import Sequential
from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras_tqdm import TQDMNotebookCallback


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from GraphConverter import GraphConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier
from ConvNetPatchy import ConvNetPatchy

### Load data

In [3]:
# Getting the data:
# Getting the data:
dataset_name = 'PTC_FM'
width = 18
receptive_field = 10
relabeling = True

PTC_FM graph tensor non exisiting at : /Users/marcelogutierrez/.gamma_link/Samples/PTC_FM/PTC_FM_patchy_tensor_w_15.npy
Create dictionary of graphs
Canonical Labeling
Getting the Neighboors 
Neighboors to Tensor
PTC_FM graph tensor created at: /Users/marcelogutierrez/.gamma_link/Samples/PTC_FM/PTC_FM_patchy_tensor_w_15.npy


In [ ]:
# Converting Graphs into Matrices:
PatchyConverter = GraphConverter(dataset_name, receptive_field, GRAPH_RELABEL_NAME)
if relabelling:
    PatchyConverter.relabel_graphs()

graph_tensor = PatchyConverter.graphs_to_Patchy_tensor()
avg_nodes_per_graph = PatchyConverter.avg_nodes_per_graph

# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)


In [5]:
val_acc = []
for j in range(1):
    # Starting Conv net with Patchy
    patchy_cnn = ConvNetPatchy(graph_tensor, graph_labels)
    patchy_cnn.split_data(random_state = j)
    patchy_cnn.build_graph()
    patchy_cnn.train_model(epochs=200)
    print('Fold {} completed with val acc : {}'.format(j,patchy_cnn.final_val_acc))
    
    val_acc.append(patchy_cnn.final_val_acc)

Fold 0 completed with val acc : 0.5428571701049805


In [6]:
print('avg val accuracy: {}'.format(np.mean(val_acc)))
print('std val accuracy: {}'.format(np.std(val_acc)))
      

avg val accuracy: 0.5428571701049805
std val accuracy: 0.0


In [ ]:
score = patchy_cnn.model.evaluate(patchy_cnn.x_test, patchy_cnn.y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(range(patchy_cnn.epochs), patchy_cnn.history.acc)
plt.legend('train data',loc='lower center')
plt.plot(range(patchy_cnn.epochs), patchy_cnn.history.val_acc)
plt.legend('test data')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()